# This notebook will be mainly used for the capstone project

In [3]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Scrapping the wikipedia page of Toronto

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url).read()
bs = BeautifulSoup(html)
table = bs.findAll('table')

In [6]:
rows = table[0].findAll('tr')

In [7]:
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])

In [8]:
for r in rows:
    elements = r.findAll('td')
    toadd = []
    for e in elements:
        toadd.append(e.text)
    if len(toadd) == 3:
        df = df.append({'PostalCode': toadd[0][0:-1], 'Borough': toadd[1][0:-1], 'Neighborhood': toadd[2][0:-1]}, ignore_index=True)

In [9]:
df = df[df['Borough'] != 'Not assigned']
df = df.reset_index(drop = True)

In [10]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df.shape

(103, 3)

# Get Latitude and the Longitude of Neighborhoods

In [38]:
# loop until you get the coordinates
lat = []
lon = []
for index, row in df.iterrows():
    location = geolocator.geocode('{}, {}, Toronto, Ontario'.format(row['Neighborhood'], row['Borough']))
    if location is None:
        location = geolocator.geocode('{}, Toronto, Ontario'.format(row['Borough']))
    lat.append(location.latitude)
    lon.append(location.longitude)


In [39]:
lat = np.asarray(lat, dtype = float)
lon = np.asarray(lon, dtype = float)
df['latitude'] = lat
df['longitude'] = lon

In [40]:
df

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.656322,-79.380916
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.656322,-79.380916
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.643556,-79.565633
99,M4Y,Downtown Toronto,Church and Wellesley,43.662693,-79.386875
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.624790,-79.393492
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.643556,-79.565633


# Explore and cluster the neighborhoods in Toronto